In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import time
import gc

from datetime import datetime

sns.set(style='whitegrid')

pd.set_option('display.max_columns', None)  

%matplotlib inline

### Feature Engineering

Usually I start a project doing data consolidation, exploratory analysis and feature engineering. However, given the data size and my computer capabillities, I'll start doing some feature engineering in specific files to generate possible significant features before merging all files together. This will reduce the consumed memory.

In [2]:
projects = pd.read_csv('../Data/Kaggle/KDDCup14/projects.csv')

In [3]:
projects = projects[['projectid', 'teacher_acctid', 'date_posted']]

In [4]:
d = {'t': 1, 'f': 0}

In [5]:
def history_features(id_key, feature_key, time_key, out_key, aggregation, df):
    """
    Create history features aggregating values by specific id and feature keys.
    id_key : the identifier key
    feature_key : the main key to be consider for aggregation
    time_key : the datetime key
    out_key : output name for the generated feature
    aggregation : aggregation to be realized (sum, count...)
    df : input dataframe
    """
    k = df[[id_key, feature_key, time_key]].drop_duplicates().copy()
    k = k.merge(k, on=[id_key], how='left')
    k = k.drop_duplicates()
    k = k[k[time_key+'_x'] > k[time_key+'_y']]
    
    d = k.groupby(by=[id_key, time_key+'_x'])[feature_key+'_y'].agg([aggregation])
    del k

    d = d.rename(columns={aggregation: out_key}).reset_index()
    y = df.merge(d, left_on=[id_key, time_key], right_on=[id_key, time_key+'_x'], how='left')
    del d
    
    y[out_key].fillna(0, inplace=True)
    y = y[[out_key, id_key, time_key]].drop_duplicates()

    return y

#### Outcomes

In [6]:
outcomes = pd.read_csv('../Data/Kaggle/KDDCup14/outcomes.csv')

In [7]:
outcomes = outcomes.merge(projects, how='right')

In [8]:
outcomes = outcomes.replace(d)

In [9]:
len(outcomes)

664098

In [10]:
outcomes.head()

,projectid,is_exciting,at_least_1_teacher_referred_donor,fully_funded,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count,teacher_acctid,date_posted
0,ffffc4f85b60efc5b52347df489d0238,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,c24011b20fc161ed02248e85beb59a90,2009-10-11
1,ffffac55ee02a49d1abc87ba6fc61135,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,57.0,0.0,7.0,947066d0af47e0566f334566553dd6a6,2011-06-11
2,ffff97ed93720407d70a2787475932b0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,100.0,0.0,3.0,462270f5d5c212162fcab11afa2623cb,2010-09-11
3,ffff418bb42fad24347527ad96100f81,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,100.0,0.0,1.0,e885fb002a1d0d39aaed9d21a7683549,2009-04-03
4,ffff2d9c769c8fb5335e949c615425eb,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,63.0,6.0,2.0,485784e52a4c85ea6783285d09d8aea4,2013-03-03


In [11]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'is_exciting', 'date_posted', 'mean_is_exciting', 'mean', outcomes), how='left')

In [12]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'at_least_1_teacher_referred_donor', 'date_posted', 'mean_at_least_1_teacher_referred_donor', 'mean', outcomes), how='left')

In [13]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'fully_funded', 'date_posted', 'mean_fully_funded', 'mean', outcomes), how='left')

In [14]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'at_least_1_green_donation', 'date_posted', 'mean_at_least_1_green_donation', 'mean', outcomes), how='left')

In [15]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'great_chat', 'date_posted', 'mean_great_chat', 'mean', outcomes), how='left')

In [16]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'one_non_teacher_referred_donor_giving_100_plus', 'date_posted', 'mean_one_non_teacher_referred_donor_giving_100_plus', 'mean', outcomes), how='left')

In [17]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'donation_from_thoughtful_donor', 'date_posted', 'mean_donation_from_thoughtful_donor', 'mean', outcomes), how='left')

In [18]:
outcomes = outcomes.merge(history_features('teacher_acctid', 'three_or_more_non_teacher_referred_donors', 'date_posted', 'mean_three_or_more_non_teacher_referred_donors', 'mean', outcomes), how='left')

In [19]:
len(outcomes)

664098

In [20]:
outcomes.columns

Index(['projectid', 'is_exciting', 'at_least_1_teacher_referred_donor',
       'fully_funded', 'at_least_1_green_donation', 'great_chat',
       'three_or_more_non_teacher_referred_donors',
       'one_non_teacher_referred_donor_giving_100_plus',
       'donation_from_thoughtful_donor', 'great_messages_proportion',
       'teacher_referred_count', 'non_teacher_referred_count',
       'teacher_acctid', 'date_posted', 'mean_is_exciting',
       'mean_at_least_1_teacher_referred_donor', 'mean_fully_funded',
       'mean_at_least_1_green_donation', 'mean_great_chat',
       'mean_one_non_teacher_referred_donor_giving_100_plus',
       'mean_donation_from_thoughtful_donor',
       'mean_three_or_more_non_teacher_referred_donors'],
      dtype='object')

In [21]:
outcomes = outcomes[['projectid', 'is_exciting', 'teacher_acctid', 'date_posted', 
                     'mean_is_exciting', 'mean_at_least_1_teacher_referred_donor', 
                     'mean_fully_funded', 'mean_at_least_1_green_donation', 'mean_great_chat',
                     'mean_one_non_teacher_referred_donor_giving_100_plus',
                     'mean_donation_from_thoughtful_donor',
                     'mean_three_or_more_non_teacher_referred_donors']]

In [22]:
outcomes.head()

,projectid,is_exciting,teacher_acctid,date_posted,mean_is_exciting,mean_at_least_1_teacher_referred_donor,mean_fully_funded,mean_at_least_1_green_donation,mean_great_chat,mean_one_non_teacher_referred_donor_giving_100_plus,mean_donation_from_thoughtful_donor,mean_three_or_more_non_teacher_referred_donors
0,ffffc4f85b60efc5b52347df489d0238,0.0,c24011b20fc161ed02248e85beb59a90,2009-10-11,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
1,ffffac55ee02a49d1abc87ba6fc61135,0.0,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
2,ffff97ed93720407d70a2787475932b0,0.0,462270f5d5c212162fcab11afa2623cb,2010-09-11,0.0,0.045455,0.6,0.772727,0.366667,0.695652,0.0,0.454545
3,ffff418bb42fad24347527ad96100f81,0.0,e885fb002a1d0d39aaed9d21a7683549,2009-04-03,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
4,ffff2d9c769c8fb5335e949c615425eb,1.0,485784e52a4c85ea6783285d09d8aea4,2013-03-03,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000


In [23]:
gc.collect()

268

In [24]:
outcomes.to_csv('../Data/Kaggle/KDDCup14/outcomes_feat.csv', index=False)

In [25]:
del outcomes

#### Donations

In [6]:
donations = pd.read_csv('../Data/Kaggle/KDDCup14/donations.csv')

In [7]:
donations = donations.merge(projects, how='right')

In [8]:
donations.drop(['donationid', 'donor_city', 'donor_state', 'donor_zip', 
                'donation_optional_support', 'donation_total', 'donation_message'], axis=1, inplace=True)

In [9]:
gc.collect()

46

In [10]:
len(donations)

3237045

In [11]:
donations.head()

,projectid,donor_acctid,is_teacher_acct,donation_timestamp,donation_to_project,dollar_amount,donation_included_optional_support,payment_method,payment_included_acct_credit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,payment_was_promo_matched,via_giving_page,for_honoree,teacher_acctid,date_posted
0,ffffac55ee02a49d1abc87ba6fc61135,22cbc920c9b5fa08dfb331422f5926b5,f,2011-08-25 14:27:34.807,42.50,10_to_100,t,no_cash_received,f,t,f,f,f,f,947066d0af47e0566f334566553dd6a6,2011-06-11
1,ffffac55ee02a49d1abc87ba6fc61135,521f1830a77c9dbbf8119d99c6206a16,f,2011-11-04 07:54:21.552,26.83,10_to_100,t,creditcard,f,t,f,f,f,f,947066d0af47e0566f334566553dd6a6,2011-06-11
2,ffffac55ee02a49d1abc87ba6fc61135,1e0a63fc8141c7ba26b8b44ca0871b90,f,2011-11-02 22:53:53.019,55.35,10_to_100,f,no_cash_received,t,f,f,f,t,f,947066d0af47e0566f334566553dd6a6,2011-06-11
3,ffffac55ee02a49d1abc87ba6fc61135,1d4acb508df29d5f1cc6d382969576cb,f,2011-11-03 23:54:01.109,8.50,10_to_100,t,paypal,f,f,f,f,f,f,947066d0af47e0566f334566553dd6a6,2011-06-11
4,ffffac55ee02a49d1abc87ba6fc61135,59c3c3cfcccc53ae855f7eee911c478b,f,2011-11-02 23:21:00.043,20.00,10_to_100,f,no_cash_received,f,f,t,f,t,f,947066d0af47e0566f334566553dd6a6,2011-06-11


In [12]:
donations = donations.merge(history_features('teacher_acctid', 'donation_to_project', 'date_posted', 'mean_donation_to_projects', 'mean', donations), how='left')

In [13]:
donations['mean_donation_to_project'] = donations.groupby('projectid')['donation_to_project'].transform('mean')

I'll create dummies for some categorical values and aggregate them. Another option would have been to take the mode. However, I'll stick with the first case.

In [14]:
donations = pd.get_dummies(donations, columns=['dollar_amount', 'payment_method', 'donation_included_optional_support', 'payment_included_acct_credit',
                                               'payment_included_campaign_gift_card', 'payment_included_web_purchased_gift_card', 
                                               'payment_was_promo_matched', 'via_giving_page', 'for_honoree', 'is_teacher_acct'])

In [15]:
donations.head()

,projectid,donor_acctid,donation_timestamp,donation_to_project,teacher_acctid,date_posted,mean_donation_to_projects,mean_donation_to_project,dollar_amount_100_and_up,dollar_amount_10_to_100,dollar_amount_under_10,payment_method_almost_home_match,payment_method_amazon,payment_method_check,payment_method_creditcard,payment_method_double_your_impact_match,payment_method_no_cash_received,payment_method_paypal,payment_method_promo_code_match,donation_included_optional_support_f,donation_included_optional_support_t,payment_included_acct_credit_f,payment_included_acct_credit_t,payment_included_campaign_gift_card_f,payment_included_campaign_gift_card_t,payment_included_web_purchased_gift_card_f,payment_included_web_purchased_gift_card_t,payment_was_promo_matched_f,payment_was_promo_matched_t,via_giving_page_f,via_giving_page_t,for_honoree_f,for_honoree_t,is_teacher_acct_f,is_teacher_acct_t
0,ffffac55ee02a49d1abc87ba6fc61135,22cbc920c9b5fa08dfb331422f5926b5,2011-08-25 14:27:34.807,42.50,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0
1,ffffac55ee02a49d1abc87ba6fc61135,521f1830a77c9dbbf8119d99c6206a16,2011-11-04 07:54:21.552,26.83,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0
2,ffffac55ee02a49d1abc87ba6fc61135,1e0a63fc8141c7ba26b8b44ca0871b90,2011-11-02 22:53:53.019,55.35,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0
3,ffffac55ee02a49d1abc87ba6fc61135,1d4acb508df29d5f1cc6d382969576cb,2011-11-03 23:54:01.109,8.50,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0
4,ffffac55ee02a49d1abc87ba6fc61135,59c3c3cfcccc53ae855f7eee911c478b,2011-11-02 23:21:00.043,20.00,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,1,0,0,1,1,0,1,0


In [16]:
dummies = ['dollar_amount_100_and_up', 'dollar_amount_10_to_100', 'dollar_amount_under_10', 
           'payment_method_almost_home_match', 'payment_method_amazon', 'payment_method_check', 'payment_method_creditcard', 
           'payment_method_double_your_impact_match', 'payment_method_no_cash_received', 'payment_method_paypal',
           'payment_method_promo_code_match', 'donation_included_optional_support_f', 'donation_included_optional_support_t',
           'payment_included_acct_credit_f', 'payment_included_acct_credit_t',
           'payment_included_campaign_gift_card_f', 'payment_included_campaign_gift_card_t',
           'payment_included_web_purchased_gift_card_f', 'payment_included_web_purchased_gift_card_t',
           'payment_was_promo_matched_f', 'payment_was_promo_matched_t',
           'via_giving_page_f', 'via_giving_page_t', 'for_honoree_f', 'for_honoree_t',
           'is_teacher_acct_f', 'is_teacher_acct_t']

for d in dummies:
    donations[d] = donations.groupby('projectid')[d].transform('sum')

In [17]:
donations.head()

,projectid,donor_acctid,donation_timestamp,donation_to_project,teacher_acctid,date_posted,mean_donation_to_projects,mean_donation_to_project,dollar_amount_100_and_up,dollar_amount_10_to_100,dollar_amount_under_10,payment_method_almost_home_match,payment_method_amazon,payment_method_check,payment_method_creditcard,payment_method_double_your_impact_match,payment_method_no_cash_received,payment_method_paypal,payment_method_promo_code_match,donation_included_optional_support_f,donation_included_optional_support_t,payment_included_acct_credit_f,payment_included_acct_credit_t,payment_included_campaign_gift_card_f,payment_included_campaign_gift_card_t,payment_included_web_purchased_gift_card_f,payment_included_web_purchased_gift_card_t,payment_was_promo_matched_f,payment_was_promo_matched_t,via_giving_page_f,via_giving_page_t,for_honoree_f,for_honoree_t,is_teacher_acct_f,is_teacher_acct_t
0,ffffac55ee02a49d1abc87ba6fc61135,22cbc920c9b5fa08dfb331422f5926b5,2011-08-25 14:27:34.807,42.50,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,6.0,1.0,0,0,0,2.0,0,4.0,1,0,3.0,4.0,6.0,1,4.0,3.0,6.0,1,7.0,0,5.0,2.0,6.0,1,7.0,0.0
1,ffffac55ee02a49d1abc87ba6fc61135,521f1830a77c9dbbf8119d99c6206a16,2011-11-04 07:54:21.552,26.83,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,6.0,1.0,0,0,0,2.0,0,4.0,1,0,3.0,4.0,6.0,1,4.0,3.0,6.0,1,7.0,0,5.0,2.0,6.0,1,7.0,0.0
2,ffffac55ee02a49d1abc87ba6fc61135,1e0a63fc8141c7ba26b8b44ca0871b90,2011-11-02 22:53:53.019,55.35,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,6.0,1.0,0,0,0,2.0,0,4.0,1,0,3.0,4.0,6.0,1,4.0,3.0,6.0,1,7.0,0,5.0,2.0,6.0,1,7.0,0.0
3,ffffac55ee02a49d1abc87ba6fc61135,1d4acb508df29d5f1cc6d382969576cb,2011-11-03 23:54:01.109,8.50,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,6.0,1.0,0,0,0,2.0,0,4.0,1,0,3.0,4.0,6.0,1,4.0,3.0,6.0,1,7.0,0,5.0,2.0,6.0,1,7.0,0.0
4,ffffac55ee02a49d1abc87ba6fc61135,59c3c3cfcccc53ae855f7eee911c478b,2011-11-02 23:21:00.043,20.00,947066d0af47e0566f334566553dd6a6,2011-06-11,0.0,29.632857,0,6.0,1.0,0,0,0,2.0,0,4.0,1,0,3.0,4.0,6.0,1,4.0,3.0,6.0,1,7.0,0,5.0,2.0,6.0,1,7.0,0.0


In [18]:
donations.drop(['donor_acctid', 'donation_timestamp', 'donation_to_project'], axis=1, inplace=True)

In [19]:
donations.drop_duplicates(inplace=True)

In [20]:
len(donations)

664098

In [26]:
donations.head()

,projectid,is_teacher_acct,teacher_acctid,date_posted,mean_donation_to_projects,mean_donation_to_project,dollar_amount_100_and_up,dollar_amount_10_to_100,dollar_amount_under_10,payment_method_almost_home_match,payment_method_amazon,payment_method_check,payment_method_creditcard,payment_method_double_your_impact_match,payment_method_no_cash_received,payment_method_paypal,payment_method_promo_code_match,donation_included_optional_support_f,donation_included_optional_support_t,payment_included_acct_credit_f,payment_included_acct_credit_t,payment_included_campaign_gift_card_f,payment_included_campaign_gift_card_t,payment_included_web_purchased_gift_card_f,payment_included_web_purchased_gift_card_t,payment_was_promo_matched_f,payment_was_promo_matched_t,via_giving_page_f,via_giving_page_t,for_honoree_f,for_honoree_t
0,ffffac55ee02a49d1abc87ba6fc61135,f,947066d0af47e0566f334566553dd6a6,2011-06-11,0.000000,29.632857,0,6.0,1.0,0,0,0,2.0,0,4.0,1,0,3.0,4.0,6.0,1,4.0,3.0,6.0,1,7.0,0,5.0,2.0,6.0,1
7,ffff97ed93720407d70a2787475932b0,f,462270f5d5c212162fcab11afa2623cb,2010-09-11,94.969254,110.002500,2,2.0,0.0,0,0,0,4.0,0,0.0,0,0,0.0,4.0,4.0,0,4.0,0.0,4.0,0,4.0,0,0.0,4.0,4.0,0
11,ffff418bb42fad24347527ad96100f81,f,e885fb002a1d0d39aaed9d21a7683549,2009-04-03,0.000000,20.500000,0,1.0,0.0,0,1,0,0.0,0,0.0,0,0,0.0,1.0,1.0,0,1.0,0.0,1.0,0,1.0,0,0.0,1.0,1.0,0
12,ffff2d9c769c8fb5335e949c615425eb,f,485784e52a4c85ea6783285d09d8aea4,2013-03-03,0.000000,52.703636,3,8.0,0.0,0,0,0,6.0,0,1.0,0,4,0.0,11.0,11.0,0,9.0,2.0,11.0,0,7.0,4,3.0,8.0,11.0,0
23,fffeebf4827d745aa36b17c2d38d1966,f,96c1a33b4f2b832595c3c6a01bc24c05,2012-12-01,0.000000,797.300000,1,0.0,0.0,0,0,0,0.0,0,1.0,0,0,0.0,1.0,0.0,1,1.0,0.0,1.0,0,1.0,0,1.0,0.0,1.0,0


In [21]:
gc.collect()

62

In [22]:
donations.to_csv('../Data/Kaggle/KDDCup14/donations_feat.csv', index=False)

In [23]:
del donations

#### Resources

In [6]:
resources = pd.read_csv('../Data/Kaggle/KDDCup14/resources.csv')

In [7]:
resources.drop(['resourceid', 'vendor_name'], axis=1, inplace=True)

In [8]:
resources.head()

,projectid,vendorid,project_resource_type,item_name,item_number,item_unit_price,item_quantity
0,ffffc4f85b60efc5b52347df489d0238,430.0,Technology,iPod nano 4th Gen 8GB (Black),249995.001,149.00,4.0
1,ffffac55ee02a49d1abc87ba6fc61135,82.0,Technology,Sony bloggie MHS-FS1 - camcorder - internal fl...,BB11216668,148.00,1.0
2,ffff97ed93720407d70a2787475932b0,767.0,Supplies,VX748 - Best-Buy Jumbo Crayons - 12-Color Box,VX748,69.95,1.0
3,ffff97ed93720407d70a2787475932b0,767.0,Supplies,LA138 - Best-Buy Write & Wipe Broad-Tip Marker...,LA138,34.95,1.0
4,ffff97ed93720407d70a2787475932b0,767.0,Supplies,BJ7471 - 1 1/2&#34; Ruled Chart Tablet,BJ7471,10.95,4.0


In [9]:
resources['item_total_price'] = resources['item_unit_price'] * resources['item_quantity']

In [10]:
resources.head()

,projectid,vendorid,project_resource_type,item_name,item_number,item_unit_price,item_quantity,item_total_price
0,ffffc4f85b60efc5b52347df489d0238,430.0,Technology,iPod nano 4th Gen 8GB (Black),249995.001,149.00,4.0,596.00
1,ffffac55ee02a49d1abc87ba6fc61135,82.0,Technology,Sony bloggie MHS-FS1 - camcorder - internal fl...,BB11216668,148.00,1.0,148.00
2,ffff97ed93720407d70a2787475932b0,767.0,Supplies,VX748 - Best-Buy Jumbo Crayons - 12-Color Box,VX748,69.95,1.0,69.95
3,ffff97ed93720407d70a2787475932b0,767.0,Supplies,LA138 - Best-Buy Write & Wipe Broad-Tip Marker...,LA138,34.95,1.0,34.95
4,ffff97ed93720407d70a2787475932b0,767.0,Supplies,BJ7471 - 1 1/2&#34; Ruled Chart Tablet,BJ7471,10.95,4.0,43.80


In [11]:
resources['unique_vendors'] = resources.groupby('projectid')['vendorid'].transform('nunique')

In [12]:
resources['unique_items'] = resources.groupby('projectid')['item_number'].transform('nunique')

In [13]:
resources['qtd_items'] = resources.groupby('projectid')['item_quantity'].transform('sum')

In [14]:
resources['total_price'] = resources.groupby('projectid')['item_total_price'].transform('sum')

In [15]:
resources['mean_price'] = resources.groupby('projectid')['item_total_price'].transform('mean')

In [16]:
resources['project_resource_type'].nunique()

6

In [17]:
resources = pd.get_dummies(resources, columns=['project_resource_type'])

In [18]:
resources.head()

,projectid,vendorid,item_name,item_number,item_unit_price,item_quantity,item_total_price,unique_vendors,unique_items,qtd_items,total_price,mean_price,project_resource_type_Books,project_resource_type_Other,project_resource_type_Supplies,project_resource_type_Technology,project_resource_type_Trips,project_resource_type_Visitors
0,ffffc4f85b60efc5b52347df489d0238,430.0,iPod nano 4th Gen 8GB (Black),249995.001,149.00,4.0,596.00,1.0,1,4.0,596.00,596.000000,0,0,0,1,0,0
1,ffffac55ee02a49d1abc87ba6fc61135,82.0,Sony bloggie MHS-FS1 - camcorder - internal fl...,BB11216668,148.00,1.0,148.00,1.0,1,1.0,148.00,148.000000,0,0,0,1,0,0
2,ffff97ed93720407d70a2787475932b0,767.0,VX748 - Best-Buy Jumbo Crayons - 12-Color Box,VX748,69.95,1.0,69.95,1.0,17,30.0,399.02,23.471765,0,0,1,0,0,0
3,ffff97ed93720407d70a2787475932b0,767.0,LA138 - Best-Buy Write & Wipe Broad-Tip Marker...,LA138,34.95,1.0,34.95,1.0,17,30.0,399.02,23.471765,0,0,1,0,0,0
4,ffff97ed93720407d70a2787475932b0,767.0,BJ7471 - 1 1/2&#34; Ruled Chart Tablet,BJ7471,10.95,4.0,43.80,1.0,17,30.0,399.02,23.471765,0,0,1,0,0,0


In [19]:
dummies = ['project_resource_type_Books', 'project_resource_type_Other', 'project_resource_type_Supplies', 
           'project_resource_type_Technology', 'project_resource_type_Trips', 'project_resource_type_Visitors']

for d in dummies:
    resources[d] = resources.groupby('projectid')[d].transform('sum')

In [20]:
resources.drop(['vendorid', 'item_name', 'item_number', 'item_unit_price', 'item_quantity', 'item_total_price'], axis=1, inplace=True)

In [21]:
resources.drop_duplicates(inplace=True)

In [22]:
resources.head()

,projectid,unique_vendors,unique_items,qtd_items,total_price,mean_price,project_resource_type_Books,project_resource_type_Other,project_resource_type_Supplies,project_resource_type_Technology,project_resource_type_Trips,project_resource_type_Visitors
0,ffffc4f85b60efc5b52347df489d0238,1.0,1,4.0,596.00,596.000000,0.0,0,0,1,0,0
1,ffffac55ee02a49d1abc87ba6fc61135,1.0,1,1.0,148.00,148.000000,0.0,0,0,1,0,0
2,ffff97ed93720407d70a2787475932b0,1.0,17,30.0,399.02,23.471765,0.0,0,17,0,0,0
19,ffff7266778f71242675416e600b94e1,1.0,1,25.0,131.25,131.250000,0.0,1,0,0,0,0
20,ffff418bb42fad24347527ad96100f81,1.0,1,1.0,694.98,694.980000,0.0,0,1,0,0,0


In [23]:
len(resources)

663773

In [25]:
gc.collect()

2226

In [26]:
resources.to_csv('../Data/Kaggle/KDDCup14/resources_feat.csv', index=False)

In [27]:
del resources

#### Essays

In [29]:
essays = pd.read_csv('../Data/Kaggle/KDDCup14/essays.csv')

In [31]:
essays.head()

,projectid,teacher_acctid,title,short_description,need_statement,essay
0,ffffc4f85b60efc5b52347df489d0238,c24011b20fc161ed02248e85beb59a90,iMath,It is imperative that teachers bring technolog...,My students need four iPods.,I am a fourth year fifth grade math teacher. T...
1,ffffac55ee02a49d1abc87ba6fc61135,947066d0af47e0566f334566553dd6a6,Recording Rockin' Readers,Can you imagine having to translate everything...,My students need a camcorder.,Can you imagine having to translate everything...
2,ffff97ed93720407d70a2787475932b0,462270f5d5c212162fcab11afa2623cb,Kindergarten In Need of Important Materials!,It takes a special person to donate to a group...,My students need 17 assorted classroom materia...,Hi. I teach a wonderful group of 4-5 year old ...
3,ffff7266778f71242675416e600b94e1,b9a8f14199e0d8109200ece179281f4f,Let's Find Out!,My Kindergarten students come from a variety o...,"My students need 25 copies of Scholastic's ""Le...",My Kindergarten students come from a variety o...
4,ffff418bb42fad24347527ad96100f81,e885fb002a1d0d39aaed9d21a7683549,Whistle While We Work!,"By using the cross curricular games requested,...",My students need grade level appropriate games...,All work and no play makes school a dull place...


In [32]:
essays['title_len'] = essays['title'].apply(lambda x: len(str(x)))
essays['essay_len'] = essays['essay'].apply(lambda x: len(str(x)))

In [33]:
essays.drop(['title', 'need_statement', 'essay'], axis=1, inplace=True)

In [34]:
essays.head()

,projectid,teacher_acctid,short_description,title_len,essay_len
0,ffffc4f85b60efc5b52347df489d0238,c24011b20fc161ed02248e85beb59a90,It is imperative that teachers bring technolog...,5,2615
1,ffffac55ee02a49d1abc87ba6fc61135,947066d0af47e0566f334566553dd6a6,Can you imagine having to translate everything...,25,1155
2,ffff97ed93720407d70a2787475932b0,462270f5d5c212162fcab11afa2623cb,It takes a special person to donate to a group...,44,1327
3,ffff7266778f71242675416e600b94e1,b9a8f14199e0d8109200ece179281f4f,My Kindergarten students come from a variety o...,15,1074
4,ffff418bb42fad24347527ad96100f81,e885fb002a1d0d39aaed9d21a7683549,"By using the cross curricular games requested,...",22,1081


In [35]:
gc.collect()

932

In [36]:
essays.to_csv('../Data/Kaggle/KDDCup14/essays_feat.csv', index=False)

#### Projects

In [26]:
projects = pd.read_csv('../Data/Kaggle/KDDCup14/projects.csv')

In [27]:
projects.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662.0,NaN,Selma Unified Sch District,Fresno,f,f,f,f,f,f,Mrs.,f,f,Literature & Writing,Literacy & Language,College & Career Prep,Applied Learning,Books,highest poverty,Grades 6-8,30.0,555.81,653.89,32.0,f,f,2014-05-12
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,d711e47810900c96f26a5d0be30c446d,4.837020e+11,32.911179,-96.723640,Dallas,TX,75243.0,urban,Richardson Ind School District,Dallas,f,f,f,f,f,f,Mrs.,f,f,Literacy,Literacy & Language,ESL,Literacy & Language,Books,highest poverty,Grades PreK-2,30.0,296.47,348.79,22.0,f,f,2014-05-12
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,665c3613013ba0a66e3a2a26b89f1b68,4.103270e+11,45.166039,-122.414576,Colton,OR,97017.0,rural,Colton School District 53,Clackamas,f,f,f,f,f,f,Mr.,f,f,Literacy,Literacy & Language,Mathematics,Math & Science,Technology,high poverty,Grades PreK-2,30.0,430.89,506.93,17.0,f,f,2014-05-11
3,bb18f409abda2f264d5acda8cab577a9,2133fc46f951f1e7d60645b0f9e48a6c,4f12c3fa0c1cce823c7ba1df57e90ccb,3.600153e+11,40.641727,-73.965655,Brooklyn,NY,11226.0,urban,New York City Dept Of Ed,Kings (Brooklyn),f,t,f,f,f,f,Mr.,t,f,Social Sciences,History & Civics,Special Needs,Special Needs,Books,highest poverty,Grades 3-5,30.0,576.07,677.73,12.0,f,f,2014-05-11
4,24761b686e18e5eace634607acbcc19f,867ff478a63f5457eaf41049536c47cd,10179fd362d7b8cf0e89baa1ca3025bb,6.227100e+10,34.043939,-118.288371,Los Angeles,CA,90006.0,urban,Los Angeles Unif Sch Dist,Los Angeles,f,f,f,f,f,f,Ms.,f,f,Mathematics,Math & Science,Literacy,Literacy & Language,Other,highest poverty,Grades PreK-2,30.0,408.40,480.47,24.0,f,f,2014-05-11


In [28]:
projects = projects.merge(history_features('schoolid', 'projectid', 'date_posted', 'qtd_projects', 'count', projects), how='left')

In [29]:
projects.drop(['schoolid', 'school_ncesid', 'school_city', 'school_state', 'school_zip', 'school_district', 'school_county'],
              axis=1, inplace=True)

In [30]:
len(projects)

664098

In [31]:
projects = projects.replace(d)

In [32]:
projects.head()

,projectid,teacher_acctid,school_latitude,school_longitude,school_metro,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,qtd_projects
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,36.576340,-119.608713,NaN,0,0,0,0,0,0,Mrs.,0,0,Literature & Writing,Literacy & Language,College & Career Prep,Applied Learning,Books,highest poverty,Grades 6-8,30.0,555.81,653.89,32.0,0,0,2014-05-12,3.0
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,32.911179,-96.723640,urban,0,0,0,0,0,0,Mrs.,0,0,Literacy,Literacy & Language,ESL,Literacy & Language,Books,highest poverty,Grades PreK-2,30.0,296.47,348.79,22.0,0,0,2014-05-12,63.0
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,45.166039,-122.414576,rural,0,0,0,0,0,0,Mr.,0,0,Literacy,Literacy & Language,Mathematics,Math & Science,Technology,high poverty,Grades PreK-2,30.0,430.89,506.93,17.0,0,0,2014-05-11,1.0
3,bb18f409abda2f264d5acda8cab577a9,2133fc46f951f1e7d60645b0f9e48a6c,40.641727,-73.965655,urban,0,1,0,0,0,0,Mr.,1,0,Social Sciences,History & Civics,Special Needs,Special Needs,Books,highest poverty,Grades 3-5,30.0,576.07,677.73,12.0,0,0,2014-05-11,572.0
4,24761b686e18e5eace634607acbcc19f,867ff478a63f5457eaf41049536c47cd,34.043939,-118.288371,urban,0,0,0,0,0,0,Ms.,0,0,Mathematics,Math & Science,Literacy,Literacy & Language,Other,highest poverty,Grades PreK-2,30.0,408.40,480.47,24.0,0,0,2014-05-11,453.0


In [33]:
gc.collect()

216

In [34]:
projects.to_csv('../Data/Kaggle/KDDCup14/projects_feat.csv', index=False)